# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233186332684                   -0.50    7.0
  2   -7.249596455229       -1.78       -1.40    1.0
  3   -7.251275425225       -2.77       -1.99    4.0
  4   -7.251180155439   +   -4.02       -2.06    5.0
  5   -7.251306304788       -3.90       -2.41    4.0
  6   -7.251338201565       -4.50       -3.18    3.0
  7   -7.251338788501       -6.23       -3.99    3.0
  8   -7.251338797126       -8.06       -4.51    1.0
  9   -7.251338798637       -8.82       -5.17    3.0
 10   -7.251338798700      -10.20       -5.66    3.0
 11   -7.251338798704      -11.42       -6.00    2.0
 12   -7.251338798705      -11.99       -6.80    3.0
 13   -7.251338798705      -13.73       -7.11    3.0
 14   -7.251338798705   +  -15.05       -7.49    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05269562924698966
[ Info: Arnoldi iteration step 2: normres = 0.4771847463013939
[ Info: Arnoldi iteration step 3: normres = 0.9214487919365693
[ Info: Arnoldi iteration step 4: normres = 0.34256346785751873
[ Info: Arnoldi iteration step 5: normres = 0.35255419847601915
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.33e-02, 4.34e-02, 2.83e-01, 2.01e-01, 1.35e-02)
[ Info: Arnoldi iteration step 6: normres = 0.426004811851776
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.49e-02, 6.53e-02, 3.83e-01, 1.31e-01, 1.08e-01)
[ Info: Arnoldi iteration step 7: normres = 0.07085571731322146
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.24e-04, 6.46e-03, 2.32e-02, 2.82e-02, 4.83e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10498948233244242
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.42e-